In [4]:
# -*- coding:utf-8 -*-
import pymongo
import etl_utils

def get_proid(comment_id):
    ss = comment_id.strip()
    i = ss.index('-')
    return ss[:i]         
    
# 插入一条数据到 result_collection
def insert_one2collection(rslt_coll, doc):
    rslt_coll.insert_one(doc)

# 删除指定collection, 当前未使用
def drop_collection(rslt_coll):
    coll_name = rslt_coll._Collection__name
    db = etl_utils.get_mongo_db()
    ret = db.drop_collection(coll_name)
    print('drop % status:' % coll_name, ret)
    
# 导出数据到result_zol_proidlist
def export2result_zol_proidlist(coll_src, coll_targ):
    coll_src_name = coll_src._Collection__name
    minid = etl_utils.get_start_id(coll_src_name)
    maxid = minid    
    cursor = coll_src.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    for cur in cursor:
        doc = { 'phoneid': '',
                'proid': cur['proid'],
                'phonename': '',
                'name': cur['title'],
                'modeltype': '',
                'comment_total':cur['comment_total'],
                'color': ''}
        maxid = cur['_id']
        insert_one2collection(coll_targ, doc)
        
    etl_utils.set_coll_export2result_log(coll_src_name, minid, maxid)


# 导出数据到result_zol_comment    
def export2result_zol_comment(coll_src, coll_targ):
    coll_src_name = coll_src._Collection__name
    minid  = etl_utils.get_start_id(coll_src_name)
    maxid = minid
    cursor = coll_src.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    for cur in cursor:
        proid = get_proid(cur['comment_id'])
        words = cur['comment_word']
        if words.strip() == '' or proid.strip() == '': continue
        doc = { 'phoneid': '',
                'proid': proid,
                'userid': cur['comment_user'],
                'userlevel': '',
                'province': '',
                'terminal': '',
                'comment_words': words,
                'comment_score': cur['comment_score'],
                'comment_date':cur['comment_date']}  
        maxid = cur['_id']
        insert_one2collection(coll_targ, doc)
        
    etl_utils.set_coll_export2result_log(coll_src_name, minid, maxid)
    
    
# 导出数据到result_zol collection
def export2result_zol(is_appenddate=False):
    print('start etl zol...')
    coll_crawl_proids = etl_utils.get_coll_crawl_proidlist('crawl_zol_proidlist')
    coll_crawl_comment = etl_utils.get_coll_crawl_comment('crawl_zol_comment')    
    coll_rstl_proids = etl_utils.get_coll_result_proidlist('result_zol_proidlist', is_appenddate)
    coll_rstl_comment = etl_utils.get_coll_result_comment('result_zol_comment', is_appenddate)
    
    export2result_zol_proidlist(coll_crawl_proids, coll_rstl_proids)
    export2result_zol_comment(coll_crawl_comment, coll_rstl_comment) 
    print("etl zol ended.")
    
# 清空zol的，etl导出result与log
def clean_zol_exported_result():
    etl_utils.clean_coll_exported_result('crawl_zol_proidlist')
    etl_utils.clean_coll_exported_result('crawl_zol_comment')
    
if __name__ == '__main__':
    is_appenddate = etl_utils.is_export_new_result_coll()
    clean_zol_exported_result()
    export2result_zol(is_appenddate)
    

please input(y/n), is export to new result collection:n
drop result_zol_proidlist_20170613tatus: {'ns': 'phone.result_zol_proidlist_20170613', 'nIndexesWas': 1, 'ok': 1.0}
drop result_zol_proidlisttatus: {'ns': 'phone.result_zol_proidlist', 'nIndexesWas': 1, 'ok': 1.0}
drop result_zol_comment_20170613tatus: {'ns': 'phone.result_zol_comment_20170613', 'nIndexesWas': 1, 'ok': 1.0}
drop result_zol_commenttatus: {'ns': 'phone.result_zol_comment', 'nIndexesWas': 1, 'ok': 1.0}
start etl zol...
result coll: result_zol_proidlist
result coll: result_zol_comment
crawl_zol_proidlist  maxid: 000000000000000000000000
crawl_zol_comment  maxid: 000000000000000000000000
etl zol ended.
